# Creating the data to test in STATA and Python

In [2]:
import numpy as np
import pandas as pd
from ujive1 import *
from ujive2 import *
from tsls import * 
from pprint import pprint

#Pick a vector length:
n = 1000

#Getting our Z's and making a Z matrix:
Z = np.random.randn(n, 1)
column_of_ones = np.ones((Z.shape[0], 1))
Z = np.hstack((column_of_ones, Z))
#pprint(Z)

#Parameter vectors:
α = np.array([1,1])
β = np.array([1,2])
#pprint(α)
#pprint(β)

#Error terms:
e1 = np.random.normal(0,5,n)
e2 = np.random.normal(0,5,n)
δ = np.random.normal(0,1)
ε = 5*e1 - 5*e2 + δ

#Making our endogenous variable:
x = np.dot(Z,α) + .2*e1
X = np.column_stack((column_of_ones, x))
#pprint(X)

#Outcome vector:
Y = np.dot(X,β) + ε

#OLS benchmark:
bhat_ols = np.dot(np.linalg.inv(np.dot(X.T,X)), np.dot(X.T, Y))

# Check to see if the Z'Z matrix is invertible
if np.linalg.matrix_rank(Z.T @ Z) == Z.shape[1]:  # Should be True
    print("Z'Z is invertible")
else:
    print("Z'Z is not invertible")

cond_number = np.linalg.cond(Z.T @ Z)
print(f"Condition number of Z.T @ Z: {cond_number}")
if cond_number > 1e10:  # Threshold for ill-conditioning
    raise ValueError("Z.T @ Z is ill-conditioned and may cause numerical instability.")

#2sls comparison:
Zt_Z = np.dot(Z.T, Z)
Zt_Z_inv = np.linalg.inv(Zt_Z)
pz = np.dot(np.dot(Z, Zt_Z_inv), Z.T)
proj_x = np.dot(pz, X)
first = np.linalg.inv(np.dot(proj_x.T, X))
second = np.dot(proj_x.T, Y)
bhat_2sls = np.dot(first, second)

#pprint(X[:,1].shape)
#pprint(Z[:,1])

ujive1 = UJIVE1(Y,X,Z,talk=False)
#jive2 = UJIVE2(Y,X,Z,talk=False)
#tsls = TSLS(Y,X,Z)

# Combine matrices into a single DataFrame
df = pd.DataFrame({
    "Y": Y,  # Outcome vector
    **{f"X{i}": X[:, i] for i in range(X.shape[1])},  # Endogenous variables
    **{f"Z{i}": Z[:, i] for i in range(Z.shape[1])}   # Instrumental variables
})

# Save the DataFrame to a CSV file
df.to_csv('data.csv', index=False)

# Print the DataFrame to verify
#print(df)


#Compare them:
print("OLS:", bhat_ols[1])
print("2SLS:", bhat_2sls[1])
print("Jive 1:", ujive1['beta'])
print(ujive1.summary(pvals= ujive1['pvals'],cis=ujive1['cis'], root_mse=ujive1['r_squared'],tstats=ujive1['tstats']))
#print("Jive 2:",jive2['beta'])

Z'Z is invertible
Condition number of Z.T @ Z: 1.0227934539030707
OLS: 14.88135287882286
2SLS: 2.9731853914281623
Jive 1: [3.17293636]

UJIVE1 Regression Results
 Coefficient  Std. Error  t-stat    P>|t|  Conf. Int. Low  Conf. Int. High
      3.1729      0.7962 [3.985] [0.0001]          1.6105           4.7354
--------------------------------------------------------------------------------
R-squared: 0.1231
Adjusted R-squared: 0.1231
F-statistic: inf
Root MSE: 0.1231
None


/Users/jonathanhyatt/weak_instruments/src/weak_instruments/ujive1.py:171: RuntimeWarning: Normally this estimator is used when Z has more columns than X. In this case Z has 2 columns and X has 2 columns.
  warnings.warn(f"Normally this estimator is used when Z has more columns than X. In this case Z has {Z.shape[1]} columns and X has {X.shape[1]} columns.", RuntimeWarning)
/Users/jonathanhyatt/weak_instruments/src/weak_instruments/ujive1.py:210: RuntimeWarning: divide by zero encountered in matmul
  P = Z @ np.linalg.inv(Z.T @ Z) @ Z.T
/Users/jonathanhyatt/weak_instruments/src/weak_instruments/ujive1.py:210: RuntimeWarning: overflow encountered in matmul
  P = Z @ np.linalg.inv(Z.T @ Z) @ Z.T
/Users/jonathanhyatt/weak_instruments/src/weak_instruments/ujive1.py:210: RuntimeWarning: invalid value encountered in matmul
  P = Z @ np.linalg.inv(Z.T @ Z) @ Z.T
/Users/jonathanhyatt/weak_instruments/src/weak_instruments/ujive1.py:211: RuntimeWarning: divide by zero encountered in matmul
  fit 